In [883]:
import json
import jieba
import pandas
import numpy as np

In [884]:
# 加载数据集
with open("./dataset/HPD/cn_train_set.json", 'r') as file:
    train_file = json.load(file)
with open("./dataset/HPD/cn_test_set.json", 'r') as file:
    test_file = json.load(file)

In [885]:
# 加载停用词表
with open('dataset/停用词表.txt', encoding='utf-8') as f:
    con = f.readlines()
    stop_words = set()
    for i in con:
        i = i.replace("\n", "")
        stop_words.add(i)

In [886]:
# 对对话词元化
train_words = {}
for segment in train_file:
    for line in train_file[segment]["对话历史"]:
        tokens = jieba.cut(line, cut_all=False)
        for token in tokens:
            # 去停用词
            if token not in stop_words and len(token) > 1:
                if token in train_words:
                    train_words[token] += 1
                else:
                    train_words[token] = 1
test_words = {}
for segment in test_file:
    for line in test_file[segment]["对话历史"]:
        tokens = jieba.cut(line, cut_all=False)
        for token in tokens:
            # 去停用词
            if token not in stop_words and len(token) > 1:
                if token in test_words:
                    test_words[token] += 1
                else:
                    test_words[token] = 1

In [887]:
train_df = pandas.DataFrame()
train_df["Word"] = train_words.keys()
train_df["Num"] = train_words.values()
test_df = pandas.DataFrame()
test_df["Word"] = test_words.keys()
test_df["Num"] = test_words.values()

In [888]:
train_df.count()

Word    11471
Num     11471
dtype: int64

In [889]:
test_df.count()

Word    2996
Num     2996
dtype: int64

In [890]:
# 构建词表
class Vocab:
    def __init__(self, tokens=None, min_freq = 0) -> None:
        if tokens is None:
            return
        # 计算频率
        freq = (tokens["Num"] / tokens["Num"].count()) * 100
        tokens["Freq"] = freq
        print(tokens["Freq"].describe())
        
        if not min_freq:
            min_freq = tokens["Freq"].min()

        # 删除频率太低的词
        tokens.drop(tokens[(tokens.Freq <= min_freq)].index, inplace=True)
        self.tokens = tokens

    def __len__(self):
        return self.tokens.count()
    
    def __getitem__(self, start, end = -1):
        if (start < 0 or start > len(self.tokens)) and \
        (end < 0 or end > len(self.tokens)):
            return
        if end == -1:
            return self.tokens.iloc[start:]
        elif not end:
            return self.tokens.iloc[start]
        return self.tokens.iloc[start: end]
    
    @property
    def unk(self):
        return 0
    
    @property
    def token_freqs(self):
        return self.tokens["Freq"]

train_table = Vocab(train_df)
test_table= Vocab(test_df)

count    11471.000000
mean         0.049012
std          0.464451
min          0.008718
25%          0.008718
50%          0.008718
75%          0.026153
max         43.300497
Name: Freq, dtype: float64
count    2996.000000
mean        0.088870
std         0.426788
min         0.033378
25%         0.033378
50%         0.033378
75%         0.066756
max        20.060080
Name: Freq, dtype: float64


In [891]:
# 输出频率前50的词
print(train_table.__getitem__(0, 50))

   Word   Num       Freq
0   弗农说   103   0.897916
1    疯子    13   0.113329
4    没完     5   0.043588
6    哈利  4967  43.300497
7    梦见     7   0.061023
8    一辆     3   0.026153
9   摩托车     5   0.043588
11   达力    57   0.496905
14   开学     8   0.069741
15  第一天     6   0.052306
16   新生     5   0.043588
18   马桶     5   0.043588
19  上楼去     2   0.017435
20  试一试     9   0.078459
21   不用    56   0.488188
22   多谢    13   0.113329
23   可怜    32   0.278964
26   脑袋    25   0.217941
27  斯梅廷     2   0.017435
29   玛姬    24   0.209223
30   病倒     2   0.017435
32   老爸     4   0.034871
33   收到    27   0.235376
34   东西   401   3.495772
35   写给     3   0.026153
36   写信    14   0.122047
37   佩妮    38   0.331270
38  德思礼    12   0.104612
39   我要   109   0.950222
40   封信     6   0.052306
41   统统    11   0.095894
42   写错     2   0.017435
43   地址     4   0.034871
44   寄给     5   0.043588
45   烧掉     2   0.017435
46   上边     2   0.017435
47   碗柜     7   0.061023
48   住嘴     2   0.017435
49   不错    70   0.610235


In [892]:
# 输出频率前50的词
print(test_table.__getitem__(0, 50))

   Word  Num       Freq
0    佩妮   36   1.201602
2    弗农    5   0.166889
3    费格    2   0.066756
4    太太    2   0.066756
6    来接    2   0.066756
7   弗农说   35   1.168224
8    玛姬    2   0.066756
11   犯傻    2   0.066756
12   讨厌   10   0.333778
13   孩子   16   0.534045
15   朋友   17   0.567423
22   哈利  601  20.060080
23   留在    5   0.166889
24   家里    9   0.300401
25   回来   12   0.400534
26   房子    4   0.133511
28  动物园    2   0.066756
29   车上    2   0.066756
34   妈妈   11   0.367156
37   达力   45   1.502003
39   警告    8   0.267023
40   小子    5   0.166889
42  一点点    3   0.100134
44   碗柜    2   0.066756
45  圣诞节    6   0.200267
47   巨蟒    2   0.066756
48   碰到    2   0.066756
50   是从    4   0.133511
51   不错   19   0.634179
52   明白   25   0.834446
53   巴西    4   0.133511
54   爬虫    2   0.066756
55  管理员    2   0.066756
56  德思礼    7   0.233645
57   这条    3   0.100134
58  决不会    3   0.100134
60   来到    3   0.100134
62   玻璃    2   0.066756
64   说话   17   0.567423
65   这是   20   0.667557
74  干什么   11   0

In [893]:
# 划分数据集
def randomly_divide_data(batch_size, original, num_steps):
    start = np.random.randint(0, num_steps - 1)
    print(original.__getitem__(start, -1))
    num_batches = 0
    X = []
    Y = []
    for i in range(0, batch_size * num_batches, batch_size):
        x_data = original.__getitem__(start + i, start + i + batch_size).index()       
        X.append(x_data)
        y_data
return X, Y
randomly_divide_data(64, train_table, 5)

3
      Word   Num       Freq
6       哈利  4967  43.300497
7       梦见     7   0.061023
8       一辆     3   0.026153
9      摩托车     5   0.043588
11      达力    57   0.496905
...    ...   ...        ...
11400   困扰     2   0.017435
11437   意愿     2   0.017435
11451   罗丝     2   0.017435
11455   泰迪     6   0.052306
11456  维克托     2   0.017435

[5254 rows x 3 columns]
